## Bloomberg BQuant Webinar Series: <br> US-China Trade War
This is a companion notebook to the `US-China Trade War: Analyzing the winners and losers using BQuant` webinar.

In [1]:
import bql
import pandas as pd  #Dataframe analytics
from collections import OrderedDict  #Ordered dictionary
from IPython.display import display  #display function

import ipywidgets as ipw  #open source widgets
import bqviz as bqv   #easy to use chart library
import bqplot as bqp  #exhaustive chart library
import bqwidgets as bqw   #bloomberg widgets

In [2]:
bq = bql.Service()  #opening BQL service

#### Downloading latest GDP level for the top 20 countries

In [3]:
univ = bq.univ.members('World', type='Region')   #new universe function for regions
gdp = bq.data.gdp(transform = 'LVL',PERIOD_TYPE='A')  #specify field GDP annual level
G20_univ = bq.univ.filter(univ, gdp.grouprank() <= 20)  #filter top 20 countries by GDP level

request = bql.Request(G20_univ, {'gdp':gdp.groupsort()})
response = bq.execute(request)

In [4]:
gdp_df = response.get('gdp').to_dataframe()  #transform single 'gdp' response into a dataframe
chart_obj = bqv.BarPlot(gdp_df[['gdp']])    #feed dataframe into a chart via bqviz

In [5]:
chart_obj.set_style().show()   #display the bqviz chart

GridBox(children=(Figure(animation_duration=500, axes=[Axis(color='white', grid_color='#3c3c3c', grid_lines='d…

#### Use an ordered dictionary to retrieve various values for G20 countries

In [6]:
fields = OrderedDict()  #Same as dictionary, however it preserves the order of the keys
fields['GDP'] = bq.data.gdp(transform = 'PCT')
fields['CPI Chg'] = bq.data.cpi(transform ='PCT', period_reference = bq.func.range('2018M1', '2019M9'), fill='prev').net_chg()
fields['Unemployment Chg']  = bq.data.unemployment(period_reference = bq.func.range('2018M1', '2019M9'), fill='prev').net_chg()
fields['REER Chg'] = bq.data.reer(period_reference = bq.func.range('2018M1', '2019M8')).pct_chg()
fields['Name'] = bq.data.id()['COUNTRY_NAME']

req = bql.Request(G20_univ, fields)

resp_names = bq.execute(req)

In [7]:
eco_data = pd.concat([x.df()[x.name] for x in resp_names], axis=1)
eco_data.set_index('Name', inplace=True)
eco_data

,GDP,CPI Chg,Unemployment Chg,REER Chg
Name,,,,
Australia,1.744580,-0.20000,-0.323949,-9.579288
Brazil,1.190000,0.03000,-0.400000,-9.334931
Canada,1.651590,0.17102,-0.400000,-2.691218
China,6.000000,1.55508,-0.290000,-2.798734
France,1.354810,-0.40000,-0.600000,-0.788063
Germany,0.500000,-0.20000,-0.400000,0.134478
India,4.500000,-1.07093,0.000000,-1.528278
Indonesia,5.017900,0.14000,-0.220000,1.851852
Italy,0.317590,-0.60000,-1.146273,-1.840239


In [8]:
chart_obj = bqv.GridPlot(eco_data.sort_values(by=['GDP']),colors=['#FFFFFF', '#1B84ED', '#CF7DFF', '#FF5A00'],plots = bqv.BarPlot,cols=2)

In [9]:
for chart in chart_obj.figure:
    chart.axes[0].tick_rotate = -45
    chart.axes[0].tick_style = {'text-anchor': 'end'}

In [10]:
chart_obj.set_style().show()

# HEATMAP ECO + sector returns

## Downloading Eco data

In [11]:
#Get name of the G20 countries and 
names = resp_names.get('Name').df()
names.index = names.index+' Country'
names.head(5)

,Name
AU Country,Australia
BR Country,Brazil
CA Country,Canada
CN Country,China
FR Country,France


In [12]:
dict_eco_fields = {'NEER':bq.data.neer().dropna(),
                   'Exports':bq.data.exports().dropna(),
                   'GDP':(1+bq.data.gdp().dropna()/100/4).cumprod()-1,
                   'PMI':bq.data.pmi().dropna(),
                   'Trade_Balance':bq.data.trade_balance().dropna(),
                   'CPI':bq.data.cpi()}

date_range = bq.func.range('2016M1,2019M10')

request = bql.Request(['US Country',  #US
                       'FR Country',  #France
                       'GB Country',  #UK
                       'CA Country',  
                       'DE Country',
                       'IN Country',
                       'AU Country',
                       'CN Country',
                       'JP Country'],
                      dict_eco_fields,
                      with_params={'PERIOD_REFERENCE':date_range})

response = bq.execute(request)

In [13]:
#Create dictionary of ECO Field >> Country time series
dict_dfs = {x : response.get(x).df().reset_index().pivot(columns = 'ID',values = x,index = 'PERIOD_REFERENCE_DATE').rename_axis(names.to_dict()['Name'],axis=1) for x in dict_eco_fields.keys()}
dict_dfs

{'CPI': ID                     Australia   Canada    China  Germany  France  \
 PERIOD_REFERENCE_DATE                                                 
 2016-01-31                   NaN  2.01126  1.75182      0.5     0.2   
 2016-02-29                   NaN  1.35566  2.28433      0.1    -0.2   
 2016-03-31                   1.3  1.26682  2.30139      0.3    -0.1   
 2016-04-30                   NaN  1.66403  2.32787     -0.1    -0.2   
 2016-05-31                   NaN  1.49724  2.03900      0.2     0.0   
 2016-06-30                   1.0  1.49372  1.87950      0.3     0.2   
 2016-07-31                   NaN  1.25687  1.80000      0.5     0.2   
 2016-08-31                   NaN  1.09976  1.33978      0.4     0.2   
 2016-09-30                   1.3  1.33753  1.90000      0.6     0.4   
 2016-10-31                   NaN  1.49372  2.09595      0.8     0.4   
 2016-11-30                   NaN  1.18018  2.25226      0.8     0.5   
 2016-12-31                   1.5  1.50197  2.07655      

In [14]:
#Normalize  all time series
dict_dfs['Trade_Balance'] = dict_dfs['Trade_Balance']-dict_dfs['Trade_Balance'].iloc[0]
dict_dfs['NEER'] = dict_dfs['NEER']/dict_dfs['NEER'].iloc[0]
dict_dfs['Exports'] = dict_dfs['Exports']-dict_dfs['Exports'].iloc[0]
dict_dfs['PMI'] = dict_dfs['PMI']/dict_dfs['PMI'].iloc[0]

In [15]:
#Create scales and axis
x_scale = bqp.DateScale()
y_scale = bqp.LinearScale()
x_axis = bqp.Axis(orientation='horizontal',scale=x_scale)
y_axis = bqp.Axis(orientation='vertical',scale=y_scale)

#Create Line plots for Eco indicators and start with GDP
line_mark = bqp.Lines(x = dict_dfs['Trade_Balance'].index,
                      y=dict_dfs['Trade_Balance'].values.T,
                      scales={'x': x_scale, 'y': y_scale},
                      display_legend=True,
                      labels = list(dict_dfs['Trade_Balance'].columns)
                      )

#Assemble the figure that will old Lines and axis together
bqplot_figure = bqp.Figure(marks=[line_mark],
                           axes=[x_axis,y_axis],
                           title = 'Trade_Balance',
                           legend_location='top-left')

#Change width layout for the figure
bqplot_figure.layout.width='1500px'
bqplot_figure

Figure(axes=[Axis(scale=DateScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'bottom'…

In [16]:
#Create interaction layer for date interval selector
brush_main = bqp.interacts.BrushIntervalSelector(scale = bqplot_figure.scale_x,
                                            marks = [bqplot_figure.marks[0]])
bqplot_figure.interaction=brush_main

## Downloading Sector Total Returns

In [17]:
country = bq.data.cntry_of_risk()
sector = bq.data.classification_name('GICS','1')
sector_string = sector.format_for_query_string().replace(' ','').upper()
list_countries = ['US','CN','FR','JP','GB','CH','CA','DE','IN','HK','AU']

In [18]:
#Creating exclusion for sector/country groups that have less than 7 securities strictly
count = bq.func.ungroup(bq.func.count(bq.func.group(bq.data.id(),
                                                    [country,sector])
                                     ),UNGROUPALLLEVELS='TRUE')

#Filter large Bloomberg bond index
univ = bq.univ.filter(bq.univ.members('BWORLD Index'),
                      country.in_(list_countries).and_(count>7)
                     )

#Use the monthly return field for given time range
ret = bq.data.day_to_day_total_return(DATES=bq.func.range('2016-01-01','2019-10-01'),
                                      per='M',
                                      fill='prev')

#Get market cap for the monthly return dates (date alignment is paramount)
mkt_cap = bq.func.applypreferences(bq.data.cur_mkt_cap(CURRENCY='USD',
                                                       fill='prev'),
                                   aligndatesby=ret['DATE'])

#Perform weighted average return by market cap for every date by country/sector grouping
wavg_ret = bq.func.wavg(bq.func.group(ret,[ret['DATE'],country,sector]),
                        bq.func.group(mkt_cap,[mkt_cap['DATE'],country,sector]))

request = bql.Request(univ,{'wavg_ret':wavg_ret})

In [19]:
response = bq.execute(request)

In [20]:
returns_df = response.get('wavg_ret').df()
returns_df

,DATE,ORIG_IDS,"DAY_TO_DAY_TOTAL_RETURN(PER=PER.M,FILL=FILL.PREV,DATES=RANGE(2016-01-01,2019-10-01)).DATE",CNTRY_OF_RISK(),"CLASSIFICATION_NAME(CLASSIFICATION_SCHEME.GICS,CLASSIFICATION_LEVEL.1)",wavg_ret
ID,,,,,,
2016-01-29T00-00-00Z:AU:Consumer Discretionary,2016-01-29,None,2016-01-29,AU,Consumer Discretionary,0.007244
2016-01-29T00-00-00Z:AU:Financials,2016-01-29,None,2016-01-29,AU,Financials,-0.091320
2016-01-29T00-00-00Z:AU:Industrials,2016-01-29,None,2016-01-29,AU,Industrials,-0.064083
2016-01-29T00-00-00Z:AU:Materials,2016-01-29,None,2016-01-29,AU,Materials,-0.115078
2016-01-29T00-00-00Z:AU:Real Estate,2016-01-29,None,2016-01-29,AU,Real Estate,0.008328
2016-01-29T00-00-00Z:CA:Communication Services,2016-01-29,None,2016-01-29,CA,Communication Services,0.019575
2016-01-29T00-00-00Z:CA:Consumer Discretionary,2016-01-29,None,2016-01-29,CA,Consumer Discretionary,-0.091691
2016-01-29T00-00-00Z:CA:Consumer Staples,2016-01-29,None,2016-01-29,CA,Consumer Staples,0.012993
2016-01-29T00-00-00Z:CA:Energy,2016-01-29,None,2016-01-29,CA,Energy,0.002211


## Charts and other widgets

In [21]:
#Create dictionary of countries >> Total return by sector
dict_countries = {}
for country in returns_df['CNTRY_OF_RISK()'].unique():
    temp_df = returns_df[returns_df['CNTRY_OF_RISK()']==country]
    dict_countries[country] = temp_df.pivot(index = 'DATE',
                                            columns = sector_string, 
                                            values = 'wavg_ret')
#Visualize first rows for Australia
dict_countries['AU'].head(5)

"CLASSIFICATION_NAME(CLASSIFICATION_SCHEME.GICS,CLASSIFICATION_LEVEL.1)",Consumer Discretionary,Financials,Industrials,Materials,Real Estate
DATE,,,,,
2016-01-29,0.007244,-0.091320,-0.064083,-0.115078,0.008328
2016-02-29,-0.018357,-0.067592,0.135358,0.101793,0.029877
2016-03-31,0.057408,0.061872,0.033044,0.072711,0.027869
2016-04-29,0.009275,0.014900,0.019643,0.186648,0.034043
2016-05-31,-0.002294,0.064128,0.019498,-0.096497,0.020444


In [22]:
#Create user defined function to calculate cumulative return between two input dates
def return_cumprod_returns_by_country(start,end):
    df = pd.concat([((1+dict_countries[x].dropna().loc[start:end]).cumprod()-1).iloc[-1] for x in list_countries],axis=1,keys=list_countries)
    return df

### Creating Heatmap figure and mark

In [23]:
#Prepare scale and data for the heatmap
col_sc = bqp.ColorScale(min=-0.2,max=0.4,mid=0.00)

#calculate initial cumulative return dataframe with start,end dates
df = return_cumprod_returns_by_country('2017-01-01','2019-10-01')

#Create the heatmap bqplot.mark and feed the above dataframe as wakeup data
gridheatmap = bqp.GridHeatMap(color=df.values ,
                              scales={'color':col_sc},
                              row = df.index,
                              column = df.columns,
                              tooltip=bqp.Tooltip(fields=['row','column','color'],
                                                  show_labels=False,
                                                  formats=['','','%.2f']))

#Add additional interactions and style features to the bqplot.mark heatmap
gridheatmap.interactions = {'click':'select', 'hover':'tooltip'}
gridheatmap.selected_style = {'opacity': '0.4'} 
gridheatmap.unselected_style={'opacity': '1.0'}

#Assemble scales, axis together for the heatmap
x_heatmap_scale,y_heatmap_scale = bqp.OrdinalScale(reverse=False),bqp.OrdinalScale(reverse=True)
x_heatmap_axis = bqp.Axis(scale=x_heatmap_scale,grid_lines='none',side='top')
y_heatmap_axis = bqp.Axis(scale=y_heatmap_scale,orientation='vertical',grid_lines='none')
color_axis = bqp.ColorAxis(scale=col_sc, num_ticks=6)

#Create X/Y Axis for the heatmap
heatmap_tile_labels = bqp.Label(x=df.columns,y=df.index,scales={'x': x_heatmap_scale, 'y': y_heatmap_scale},align='middle',colors=['black'])

#Assemble heatmap mark into a bqplot figure
heatmap_figure = bqp.Figure(title = 'Return matrix',marks=[gridheatmap,heatmap_tile_labels])
heatmap_figure.axes = [x_heatmap_axis,
                        y_heatmap_axis,
                        color_axis]
heatmap_figure.fig_margin= {'top':80, 'bottom':60, 'left':160, 'right':60}
heatmap_figure

Figure(axes=[Axis(grid_lines='none', scale=OrdinalScale(), side='top'), Axis(grid_lines='none', orientation='v…

In [24]:
selector = ipw.Dropdown(options=dict_eco_fields.keys(),value='Trade_Balance')
selector

Dropdown(index=5, options=('PMI', 'CPI', 'Exports', 'NEER', 'GDP', 'Trade_Balance'), value='Trade_Balance')

### Creating Callback functions for interactivity logic

In [25]:
#User defined function to calculate cumulative return and display proper return table to the heatmap
def on_index_select(evt):
    new = evt
    dates = (dict_dfs[selector.value].iloc[new['new']['selected']].index[0].strftime('%Y-%m-%d'),
             dict_dfs[selector.value].iloc[new['new']['selected']].index[-1].strftime('%Y-%m-%d'))
    #Calculate new cumulative returns
    temp_df = return_cumprod_returns_by_country(dates[0],dates[1])
    gridheatmap.color = temp_df
    gridheatmap.row = temp_df.index
    gridheatmap.column = temp_df.columns
    heatmap_figure.title = 'Total Return {dates}'.format(dates=str(dates))
    #Push 14 top country/sector in a table
    grid_data = pd.DataFrame(temp_df.stack()).reset_index().nlargest(14,0).set_index(['level_1','level_0']).rename_axis({0:'TR'},axis=1).reset_index()
    grid_data['TR'] = round(grid_data['TR']*100,2)
    DGrid_top_14.data=grid_data

#Attach the callback function to the ECO line mark
line_mark.observe(on_index_select)

In [26]:
#User defined function for the ECO dropdown to feed the appropriate dataset to the Line chart
def callback_change_data(dd):
    line_mark.unobserve_all()
    line_mark.x = dict_dfs[selector.value].index
    line_mark.y = dict_dfs[selector.value].values.T
    bqplot_figure.title = selector.value
    line_mark.observe(on_index_select)

#Attach the callback function to the ECO dropdown
selector.observe(callback_change_data)

In [27]:
#Button for Equity transparency request
button_equities = ipw.Button(description = 'Retrieve Equities')
button_equities

Button(description='Retrieve Equities', style=ButtonStyle())

In [28]:
#User defined function to retrieve the company transparency for the selected country/sector tile from the heatmap, upon button click
def callback_bql_company(btn):
    selected_sector = gridheatmap.row[gridheatmap.selected[0][0]]
    selected_country = gridheatmap.column[gridheatmap.selected[0][1]]
    label_status.value = 'Requesting {sector} & {country}'.format(sector=selected_sector,country=selected_country)
    dates = (dict_dfs[selector.value].iloc[line_mark.selected].index[0].strftime('%Y-%m-%d'),dict_dfs[selector.value].iloc[line_mark.selected].index[-1].strftime('%Y-%m-%d'))
    univ = bq.univ.members('BWORLD Index',DATES=dates[1])
    mkt_cap = bq.data.cur_mkt_cap(CURRENCY='USD',fill='prev',DATES=dates[1])
    filtered_univ = bq.univ.filter(bq.univ.filter(univ,(bq.data.gics_sector_name()==selected_sector).and_(bq.data.cntry_of_risk()==selected_country)),mkt_cap.grouprank()<=10)
    total_return = bq.data.total_return(calc_interval=bq.func.range(dates[0],dates[1]))
    name = bq.data.name()
    request = bql.Request(filtered_univ,{'name':name,'TR':total_return,'MCap':mkt_cap})
    response = bq.execute(request)
    df = pd.concat([x.df()[x.name] for x in response],axis=1).reset_index()
    df_to_push = df.set_index('ID').sort_values('MCap',ascending=False).reset_index()
    df_to_push['TR'] = round(df_to_push['TR']*100,2)
    df_to_push['MCap'] = round(df_to_push['MCap']/1e9,2)
    DGrid_equities.data = df_to_push
    label_status.value = 'Request Done'

#Attach the callback function to the Equity request button
button_equities.on_click(callback_bql_company)

### Creating Datagrid to display data tables

In [29]:
#Create columns for both datagrids
DGrid_top_14 = bqw.DataGrid(column_defs = [{'field': 'level_1', 'filter': 'text', 'headerName': 'Country', 'width': 80},
                                           {'field': 'level_0', 'filter': 'text', 'headerName': 'Sector', 'width': 120},
                                           {'field': 'TR', 'filter': 'text', 'headerName': 'TR %', 'width': 80}])

DGrid_equities = bqw.DataGrid(column_defs = [{'field': 'ID', 'filter': 'text', 'headerName': 'Ticker', 'width': 100},
                                             {'field': 'TR', 'filter': 'text', 'headerName': 'TR %', 'width': 80},
                                             {'field': 'MCap', 'filter': 'text', 'headerName': 'MCap', 'width': 100},
                                             {'field': 'name', 'filter': 'text', 'headerName': 'Name', 'width': 120}])
display(DGrid_top_14,DGrid_equities)

DataGrid(column_defs=[{'width': 80, 'field': 'level_1', 'headerName': 'Country', 'filter': 'text'}, {'width': …

DataGrid(column_defs=[{'width': 100, 'field': 'ID', 'headerName': 'Ticker', 'filter': 'text'}, {'width': 80, '…

In [30]:
#Label status when requesting equity transparency
label_status = ipw.Label(value='')

In [31]:
#Title for the top 14 datagrid
lbl = ipw.Label(value = 'Top 14 tiles')

### Leveraging on HBox/VBox/GridBox to create dashboard of the above widgets

In [32]:
#Create Vertical and Horizontal boxes to hold the datagrid widgets
box_top_14 = ipw.VBox([lbl,DGrid_top_14],layout=ipw.Layout(width='auto',height='auto'))
box_equities = ipw.VBox([ipw.HBox([button_equities,label_status]),DGrid_equities],layout=ipw.Layout(width='auto',height='auto'))

In [33]:
#Layout changes for figures to fit properly into the Ipywidgets.Gridbox
bqplot_figure.layout.width='auto'
bqplot_figure.layout.height='auto'

heatmap_figure.layout.width='auto'
heatmap_figure.layout.height='auto'

selector.layout.grid_area='selector'
bqplot_figure.layout.grid_area='bqplot_figure'
heatmap_figure.layout.grid_area='heatmap_figure'
box_top_14.layout.grid_area='box_top_14'
box_equities.layout.grid_area='box_equities'


In [34]:
#Create Gridbox to hold all the widget into a dashboard. see following link to learn more 
#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-Grid-layout
gbox = ipw.GridBox([selector,bqplot_figure,heatmap_figure, box_top_14,box_equities],layout=ipw.Layout(grid_template_rows = '30px 500px 500px',
                                                                                     grid_template_columns = '300px 300px 300px 400px',
                                                                                     grid_template_areas = '''"selector . . ."
                                                                                     "bqplot_figure bqplot_figure bqplot_figure bqplot_figure"
                                                                                     "heatmap_figure heatmap_figure box_top_14 box_equities"
                                                                                     '''))
gbox

GridBox(children=(Dropdown(index=5, layout=Layout(grid_area='selector'), options=('PMI', 'CPI', 'Exports', 'NE…